In [1]:
#Importing Libraries
import tensorflow as tf #open source library for machine learning and it is end to end
from tensorflow.keras.utils import img_to_array,array_to_img,load_img #Public Keras utilities
from skimage.io import imsave #Skimage reads the image, converts it from JPEG into a NumPy array, and returns the array
import numpy as np #for working with arrays
from skimage import color
import os #for opening files etc
import random
from PIL import Image

In [2]:
#Getting Images

X = []
for imagename in os.listdir('Dataset/Train/'):#takes the filename of all images
    X.append(img_to_array(load_img('Dataset/Train/'+imagename)))#takes every image in directory and loads it and then convert to array and add it into the list
X = np.array(X, dtype=float) #converts X which is a list to array and changes datatype to float


# Set up train and test data

split = int(0.95*len(X)) #splitting the data 
Xtrain = X[:split] 
Xtrain = 1.0/255*Xtrain #normalization of training data to standardize data and it improves the accuracy and integrity of your data

In [3]:
#CNN model

from tensorflow.keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from tensorflow.keras.layers import Activation, Dense, Dropout, Flatten, InputLayer
from tensorflow.keras.layers import BatchNormalization
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.models import Sequential

model = Sequential() #Sequential is the easiest way to build a model in Keras. It allows you to build a model layer by layer.one input one output

#Input Layer

model.add(Conv2D(64, (3, 3), input_shape=(256, 256, 1), activation='relu', padding='same'))#input shape 3 tuples so 3 dimensions 256,256 is the pixels


#Hidden Layers
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))#rectified linear is the most used activation function for all CNN models.we use relu because it helps prevent expotential growth in computation.
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))#Conv2D is convolution layer this layer creates a convolution kernel
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))# the number of pixels shifts over the input matrix. When the stride is 1 then we move the filters to 1 pixel at a time.
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))#padding can be either valid or same. same means padding with zeros evenly
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))#512 is the numbers of filters that convolutional layers will learn from.(3,3) is the kernel size the number must be odd
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))#a simple scaling up of the image by using nearest neighbour or bilinear upsampling, so nothing smart. Advantage is it's cheap.(2,2) is the size indicating no. of columns and rows.
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='softmax', padding='same'))
model.add(UpSampling2D((2, 2)))
#Compiling the CNN

model.compile(optimizer='adam', loss='mse', metrics = ['accuracy'])#RMSProp uses the second moment by with a decay rate to speed up from AdaGrad. Adam uses both first and second moments of the gradient, and is generally the best choice
#we tried adam and rmsprop and sgd adam is better than sgd for cnn as its more efficent
#loss is measured with mean squared error
#metrics used to judge performance of model

In [4]:
import matplotlib.pyplot as plt

# Image transformer
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20,
    horizontal_flip=True)#image data generation takes the original data and transforms based on parameters passed and adds this data to the original data and runs this.This is called data augmention.

# Generate training data
batch_size = 10
def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = color.rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

# Train model      
tensorboard = TensorBoard(log_dir="/output/beta_run")
trainedmodel = model.fit(image_a_b_gen(batch_size), callbacks=[tensorboard],epochs=500, steps_per_epoch=30)


# summarize history for accuracy
plt.plot(trainedmodel.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(trainedmodel.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Epoch 1/500
30/30 [==============================] - 55s 2s/step - loss: 0.2349 - accuracy: 0.5540
Epoch 2/500
30/30 [==============================] - 54s 2s/step - loss: 0.2224 - accuracy: 0.5596
Epoch 3/500
 6/30 [=====>........................] - ETA: 48s - loss: 0.2208 - accuracy: 0.5550

In [ ]:
# Save model

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")

In [ ]:
# load json and create model

from keras.models import model_from_json
json_file = open('trainedmodel.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("trainedmodel.h5")

In [ ]:
loaded_model.summary()

In [ ]:
# Manual Testing of Images

import cv2
import os
%matplotlib inline 
from matplotlib import pyplot as plt

fig, ax = plt.subplots(23, 2, figsize=(16,100))
row = 0
colorize = []
print('Output of the Model')
for filename in os.listdir('Dataset/Test/'):
	image = Image.open('Dataset/Test/'+filename)
	new_image = image.resize((256, 256))
	new_image.save('Dataset/Test/'+filename)
	colorize.append(img_to_array(load_img('Dataset/Test/'+filename)))
	ax[row,0].imshow(load_img('Dataset/Test/'+filename), interpolation='nearest')
	row +=1

colorize = np.array(colorize, dtype=float)
colorize = color.rgb2lab(1.0/255*colorize)[:,:,:,0]
colorize = colorize.reshape(colorize.shape+(1,))


# Test model
output = loaded_model.predict(colorize)
output = output * 128

row = 0

# Output colorizations
for i in range(len(output)):
	cur = np.zeros((256, 256, 3))
	cur[:,:,0] = colorize[i][:,:,0]
	cur[:,:,1:] = output[i]
	resImage = color.lab2rgb(cur)
	ax[row,1].imshow(resImage, interpolation='nearest')
	row += 1
    

# 	imsave("result/img_"+str(i)+".png", resImage)
    
    